In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/freedoom2-wad/freedoom2.wad


In [14]:
import sys
sys.path.append('/kaggle/input/vizdoom')  # donde quede el dataset


In [15]:
pip install vizdoom --pre

Note: you may need to restart the kernel to use updated packages.


In [16]:
# CELDA 1: Imports (sin cambios)
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import sys
sys.path.append('/kaggle/input/vizdoom')

!pip install vizdoom --pre

from vizdoom import *
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import imageio
import seaborn as sns
from IPython.display import Image

/kaggle/input/freedoom2-wad/freedoom2.wad


In [17]:
# CELDA 2: DoomFreedoomEnv — VERSIÓN "NO COWARDS" (Anti-Retroceso)
import numpy as np
import cv2
from vizdoom import *
from gym import spaces

class DoomFreedoomEnv:
    def __init__(self):
        self.game = DoomGame()
        self.game.set_doom_game_path("/kaggle/input/freedoom2-wad")
        self.game.set_doom_scenario_path("/kaggle/input/freedoom2-wad")
        self.game.set_doom_map("MAP01")

        self.game.set_window_visible(False)
        self.game.set_sound_enabled(False)
        self.game.set_screen_format(ScreenFormat.GRAY8)
        self.game.set_screen_resolution(ScreenResolution.RES_320X240)

        # Acciones: 0:FWD, 1:BACK, 2:LEFT, 3:RIGHT, 4:ATTACK
        self.actions = [
            [1, 0, 0, 0, 0], 
            [0, 1, 0, 0, 0], 
            [0, 0, 1, 0, 0], 
            [0, 0, 0, 1, 0], 
            [0, 0, 0, 0, 1], 
        ]
        
        self.buttons = [Button.MOVE_FORWARD, Button.MOVE_BACKWARD, Button.TURN_LEFT, Button.TURN_RIGHT, Button.ATTACK]
        for b in self.buttons: self.game.add_available_button(b)

        for v in [GameVariable.POSITION_X, GameVariable.POSITION_Y, GameVariable.ANGLE, GameVariable.KILLCOUNT, GameVariable.HEALTH]:
            self.game.add_available_game_variable(v)

        # Living reward negativo muy pequeño para forzar movimiento constante
        self.game.set_living_reward(-0.0001) 
        
        # 8400 ticks = tiempo suficiente para cruzar el mapa
        self.game.set_episode_timeout(8400) 
        
        self.game.set_death_penalty(1.0)
        self.game.set_mode(Mode.PLAYER)
        self.game.init()

        self.prev_x = 0; self.prev_y = 0; self.prev_kills = 0; self.prev_health = 0
        self.action_space_size = len(self.actions)

    def _get_frame(self):
        state = self.game.get_state()
        if state is None: return np.zeros((84, 84), dtype=np.uint8)
        return cv2.resize(state.screen_buffer, (84, 84)).astype(np.uint8)

    def reset(self):
        self.game.new_episode()
        state = self.game.get_state()
        v = state.game_variables
        self.prev_x, self.prev_y = v[0], v[1]
        self.prev_kills = v[3]; self.prev_health = v[4]
        return self._get_frame()

    def step(self, action_id):
        # action_id 0 = FWD, 1 = BACK
        reward_base = self.game.make_action(self.actions[action_id], 4)
        done = self.game.is_episode_finished()
        state = self.game.get_state()
        
        if state is None or state.game_variables[4] <= 0:
            return np.zeros((84, 84), dtype=np.uint8), -1.0, True

        x, y, angle, kills, health = state.game_variables
        reward = 0.0
        
        # --- REWARD SHAPING CORREGIDO ---
        
        # 1. Kills: Lo más importante
        if kills > self.prev_kills: 
            reward += 1.0 
        
        # 2. Movimiento inteligente
        # Penalizar marcha atrás (Acción 1)
        if action_id == 1:
            reward -= 0.01  # Pequeño castigo por retroceder (solo úsalo si es urgente)
        
        # Incentivar ir hacia adelante (Acción 0) si realmente se mueve
        dist = np.sqrt((x - self.prev_x)**2 + (y - self.prev_y)**2)
        if action_id == 0 and dist > 1.0:
            reward += 0.002 # Pequeño premio por avanzar
            
        # 3. Penalizar daño
        if health < self.prev_health: 
            reward -= 0.05

        self.prev_x, self.prev_y = x, y
        self.prev_kills = kills
        self.prev_health = health

        return self._get_frame(), np.clip(reward, -1.0, 1.0), done

In [18]:
# CELDA 3: Instancia y plot (sin cambios)
env = DoomFreedoomEnv()
frame = env.reset()
print("Raw frame shape:", frame.shape)
plt.imshow(frame, cmap='gray')
plt.title("Freedoom MAP01 — Frame inicial")
plt.axis("off")
plt.show()

FileDoesNotExistException: File "/kaggle/input/freedoom2-wad(.wad)" does not exist.

In [ ]:
# CELDA 4: FrameStack (sin cambios)
class FrameStack:
    def __init__(self, k=4):
        self.k = k
        self.frames = deque([], maxlen=k)

    def reset(self, frame):
        self.frames = deque([frame]*self.k, maxlen=self.k)
        return np.stack(self.frames, axis=0)

    def step(self, frame):
        self.frames.append(frame)
        return np.stack(self.frames, axis=0)

In [ ]:
# CELDA 5: ARQUITECTURA RECURRENTE (CNN + GRU)
import torch
import torch.nn as nn
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def init_layer(layer, std=np.sqrt(2), bias_const=0.0):
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)
    return layer

class RecurrentActorCritic(nn.Module):
    def __init__(self, input_shape, action_dim):
        super().__init__()
        
        # 1. Feature Extractor (CNN) - Igual que antes
        self.conv = nn.Sequential(
            init_layer(nn.Conv2d(4, 32, 8, stride=4)), nn.ReLU(),
            init_layer(nn.Conv2d(32, 64, 4, stride=2)), nn.ReLU(),
            init_layer(nn.Conv2d(64, 64, 3, stride=1)), nn.ReLU()
        )

        with torch.no_grad():
            dummy = torch.zeros(1, *input_shape)
            out = self.conv(dummy)
            self.flat_size = out.view(1, -1).shape[1]
            print(f"✔ CNN Output Size: {self.flat_size}")

        # 2. MEMORIA RECURRENTE (GRU)
        # Recibe los features de la CNN y la memoria anterior
        self.gru_hidden_size = 512
        self.gru = nn.GRU(self.flat_size, self.gru_hidden_size, batch_first=True)
        
        # 3. Cabezas Actor y Critic
        self.actor = init_layer(nn.Linear(self.gru_hidden_size, action_dim), std=0.01)
        self.critic = init_layer(nn.Linear(self.gru_hidden_size, 1), std=1.0)
        
        # Red auxiliar para RND (Feature Extractor base)
        # Necesitamos una forma de extraer features sin pasar por la GRU para el RND
        self.fc_rnd = nn.Linear(self.flat_size, 512)

    def forward(self, x, hidden=None):
        # x shape: (Batch, 4, 84, 84)
        
        # Normalización
        if x.max() > 1.0: x = x.float() / 255.0
            
        # CNN
        features = self.conv(x)
        features = features.view(features.size(0), -1) # Flatten (Batch, flat_size)
        
        # GRU Logic
        # La GRU espera (Batch, Seq_Len, Features). Aquí Seq_Len=1 porque vamos paso a paso.
        features_seq = features.unsqueeze(1) 
        
        if hidden is None:
            # Si es el primer paso, memoria en ceros
            hidden = torch.zeros(1, x.size(0), self.gru_hidden_size).to(device)
            
        # Forward GRU
        gru_out, new_hidden = self.gru(features_seq, hidden)
        
        # Quitamos la dimensión de secuencia para las capas finales
        gru_out_flat = gru_out[:, -1, :]
        
        return self.actor(gru_out_flat), self.critic(gru_out_flat), new_hidden

    def get_features_for_rnd(self, x):
        # Función auxiliar solo para RND (ignora la memoria temporal)
        if x.max() > 1.0: x = x.float() / 255.0
        f = self.conv(x)
        f = f.view(f.size(0), -1)
        return torch.relu(self.fc_rnd(f))

In [ ]:
# CELDA 6: RND OPTIMIZADO (Estable y Determinista)
class RND(nn.Module):
    def __init__(self, feat_dim=512):
        super().__init__()
        
        # Redujimos un poco la dimensión oculta para forzar compresión de información
        dim_h = 128 
        
        # TARGET: Red aleatoria fija. 
        # ¡CRÍTICO: SIN DROPOUT! El target debe ser determinista.
        # Usamos init_layer (definida en Celda 5) para asegurar buena distribución inicial.
        self.target = nn.Sequential(
            init_layer(nn.Linear(feat_dim, dim_h)), 
            nn.LeakyReLU(), # LeakyReLU es mejor para evitar neuronas muertas en el target
            init_layer(nn.Linear(dim_h, dim_h))
        )
        
        # PREDICTOR: Intenta imitar al target.
        # Hacemos el predictor un poco más profundo para que tenga capacidad de aprender
        self.predictor = nn.Sequential(
            init_layer(nn.Linear(feat_dim, dim_h)), 
            nn.LeakyReLU(),
            init_layer(nn.Linear(dim_h, dim_h)), 
            nn.LeakyReLU(),
            init_layer(nn.Linear(dim_h, dim_h))
        )
        
        # Congelamos el Target (nunca se entrena)
        for p in self.target.parameters():
            p.requires_grad = False

    def forward(self, feat):
        # IMPORTANTE: feat viene de la CNN principal.
        # Debemos usar .detach() para que el entrenamiento del RND 
        # NO modifique los pesos de la CNN (Feature Extractor).
        # La CNN debe aprender a ver enemigos, no a facilitar la vida del RND.
        
        feat_in = feat.detach() 
        
        t = self.target(feat_in)
        p = self.predictor(feat_in)
        return p, t

In [ ]:
# CELDA 7: PPO UPDATE (Soporte Recurrente)
def ppo_update(model, rnd, optM, optR, data, clip=0.2, epoch=4, batch=64):
    # Ahora recibimos también 'hiddens' en la data
    obs, acts, old_logp, returns, adv, hiddens = data
    
    obs = torch.tensor(obs, dtype=torch.float32).to(device)
    acts = torch.tensor(acts, dtype=torch.long).to(device)
    old_logp = torch.tensor(old_logp, dtype=torch.float32).to(device)
    returns = torch.tensor(returns, dtype=torch.float32).to(device)
    adv = torch.tensor(adv, dtype=torch.float32).to(device)
    
    # El hidden state almacenado (Batch, 1, 512) -> Transponer a (1, Batch, 512) para GRU
    hiddens = torch.tensor(hiddens, dtype=torch.float32).to(device).permute(1, 0, 2).contiguous()

    adv = (adv - adv.mean()) / (adv.std() + 1e-8)
    dataset_size = len(obs)
    total_loss_val = 0
    
    for _ in range(epoch):
        idxs = np.random.permutation(dataset_size)
        
        for start in range(0, dataset_size, batch):
            b = idxs[start:start + batch]
            
            # Recortamos el hidden state correspondiente a este batch
            # Shape necesaria: (1, Batch_Size, 512)
            batch_hidden = hiddens[:, b, :] 
            
            # --- FORWARD CON MEMORIA ---
            # Pasamos el hidden state guardado. 
            # Nota: Esto entrena el "paso actual" condicionando al recuerdo anterior.
            logits, values, _ = model(obs[b], batch_hidden)
            values = values.squeeze()

            if torch.isnan(logits).any(): continue

            dist = torch.distributions.Categorical(logits=logits)
            new_logp = dist.log_prob(acts[b])
            entropy = dist.entropy().mean()

            ratio = torch.exp(new_logp - old_logp[b])
            surr1 = ratio * adv[b]
            surr2 = torch.clamp(ratio, 1.0 - clip, 1.0 + clip) * adv[b]
            actor_loss = -torch.min(surr1, surr2).mean()
            
            critic_loss = 0.5 * ((returns[b] - values) ** 2).mean()

            # Mantenemos Entropía Alta (0.05) como acordamos
            loss = actor_loss + critic_loss - 0.05 * entropy 

            optM.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optM.step()
            total_loss_val += loss.item()

            # --- RND UPDATE ---
            feat_rnd = model.get_features_for_rnd(obs[b]) # Usamos la función auxiliar
            p, t = rnd(feat_rnd.detach())
            rnd_loss = ((p - t) ** 2).mean()
            optR.zero_grad()
            rnd_loss.backward()
            torch.nn.utils.clip_grad_norm_(rnd.parameters(), 0.5)
            optR.step()

    return total_loss_val / (epoch * (dataset_size // batch))

In [ ]:
# CELDA 8: BUCLE DE ENTRENAMIENTO (CON MEMORIA LSTM/GRU)
import torch.optim as optim
from collections import deque
import os

# --- CONFIGURACIÓN ---
MAX_EPISODES = 5000      
BATCH_SIZE = 64          
LR_ACTOR = 1e-4          
LR_RND = 1e-4
GAMMA = 0.99
GAE_LAMBDA = 0.95
SAVE_INTERVAL = 100       

env = DoomFreedoomEnv() # Asegúrate de tener la versión Celda 2 "NO COWARDS" cargada
action_dim = env.action_space_size

# Instanciamos el modelo RECURRENTE
agent = RecurrentActorCritic((4, 84, 84), action_dim).to(device)
rnd_model = RND(feat_dim=512).to(device) # RND usa el output size de la fc_rnd

opt_agent = optim.Adam(agent.parameters(), lr=LR_ACTOR)
opt_rnd = optim.Adam(rnd_model.parameters(), lr=LR_RND)
sched_agent = optim.lr_scheduler.StepLR(opt_agent, step_size=1000, gamma=0.9)

print(f"🧠 Iniciando Agente Recurrente (GRU) por {MAX_EPISODES} episodios.")

loss_history, reward_history, kill_history = [], [], []

for ep in range(MAX_EPISODES):
    # Buffer ahora incluye 'b_hiddens'
    b_obs, b_acts, b_logp, b_rews, b_vals, b_int_rews, b_hiddens = [], [], [], [], [], [], []
    
    frame = env.reset()
    state_stack = deque([frame] * 4, maxlen=4)
    
    # INICIALIZAR MEMORIA (1, 1, 512)
    hidden = torch.zeros(1, 1, 512).to(device)
    
    done = False
    ep_reward = 0
    ep_kills = 0 
    
    while not done:
        state_np = np.array(state_stack)
        state_tensor = torch.tensor(state_np, dtype=torch.float32).unsqueeze(0).to(device)
        
        with torch.no_grad():
            # Pasamos el estado y la memoria
            logits, val, new_hidden = agent(state_tensor, hidden)
            
            dist = torch.distributions.Categorical(logits=logits)
            action = dist.sample()
            logp = dist.log_prob(action)
            
            # RND sobre features planos
            feat = agent.get_features_for_rnd(state_tensor)
            p, t = rnd_model(feat)
            int_reward = ((p - t) ** 2).mean().item()

        next_frame, reward, done = env.step(action.item())
        
        if reward >= 0.9: ep_kills += 1

        # Guardamos datos + HIDDEN STATE ACTUAL (Crucial: detach para romper grafo)
        b_obs.append(state_np)
        b_acts.append(action.item())
        b_logp.append(logp.item())
        b_rews.append(reward)
        b_int_rews.append(int_reward)
        b_vals.append(val.item())
        # Guardamos la memoria que generó esta acción (cpu numpy para ahorrar VRAM)
        b_hiddens.append(hidden.cpu().numpy().squeeze(0)) 

        # Actualizamos para el siguiente paso
        hidden = new_hidden
        ep_reward += reward
        state_stack.append(next_frame)
        
    # --- UPDATE (Igual que antes pero pasando b_hiddens) ---
    b_obs = np.array(b_obs)
    b_acts = np.array(b_acts)
    b_logp = np.array(b_logp)
    b_vals = np.array(b_vals)
    b_rews = np.array(b_rews)
    b_int_rews = np.array(b_int_rews)
    b_hiddens = np.array(b_hiddens) # Shape (Steps, 1, 512)
    
    # RND Norm
    if np.std(b_int_rews) > 0:
        b_int_rews = (b_int_rews - np.mean(b_int_rews)) / np.std(b_int_rews)
    b_int_rews = np.clip(b_int_rews, -1, 1) * 0.1
    
    total_rewards = b_rews + b_int_rews
    returns = np.zeros_like(total_rewards); adv = np.zeros_like(total_rewards); lastgaelam = 0
    
    for t in reversed(range(len(b_rews))):
        if t == len(b_rews) - 1: nextnonterminal = 0.0; nextvalues = 0.0
        else: nextnonterminal = 1.0; nextvalues = b_vals[t+1]
        delta = total_rewards[t] + GAMMA * nextvalues * nextnonterminal - b_vals[t]
        adv[t] = lastgaelam = delta + GAMMA * GAE_LAMBDA * nextnonterminal * lastgaelam
    returns = adv + b_vals
    
    if len(b_obs) >= BATCH_SIZE:
        # Pasamos b_hiddens a ppo_update
        loss = ppo_update(agent, rnd_model, opt_agent, opt_rnd, 
                          (b_obs, b_acts, b_logp, returns, adv, b_hiddens), 
                          batch=BATCH_SIZE)
        loss_history.append(loss)
    else: loss = 0

    sched_agent.step()
    reward_history.append(ep_reward); kill_history.append(ep_kills)
    
    if ep % 10 == 0:
        avg_r = np.mean(reward_history[-10:]) if len(reward_history) > 0 else 0
        avg_k = np.mean(kill_history[-10:]) if len(kill_history) > 0 else 0
        print(f"[EP {ep}] Rew: {ep_reward:.2f} (Avg: {avg_r:.2f}) | Kills: {ep_kills} (Avg: {avg_k:.1f}) | Loss: {loss:.4f}")
    
    if ep > 0 and ep % SAVE_INTERVAL == 0:
        torch.save(agent.state_dict(), "doom_agent_gru.pth")
        print(f"💾 Checkpoint GRU guardado en ep {ep}")

In [ ]:
# CELDA 9: Visualización de Resultados (Versión GRU)
import matplotlib.pyplot as plt
import numpy as np

def plot_rewards(rewards, kills=None):
    plt.figure(figsize=(12, 5))
    
    # --- GRÁFICA 1: REWARDS ---
    plt.subplot(1, 2, 1)
    plt.plot(rewards, label='Reward Crudo', color='dodgerblue', alpha=0.3)
    
    if len(rewards) >= 50:
        # Media móvil más larga para ver tendencia en 5000 episodios
        moving_avg = np.convolve(rewards, np.ones(50)/50, mode='valid')
        plt.plot(range(49, len(rewards)), moving_avg, color='navy', linewidth=2, label='Tendencia (Media 50 ep)')
    
    plt.xlabel("Episodio")
    plt.ylabel("Reward Total")
    plt.title("Entrenamiento Recurrente (PPO + GRU + RND)")
    plt.legend()
    plt.grid(True, alpha=0.3)

    # --- GRÁFICA 2: KILLS ---
    if kills is not None and len(kills) > 0:
        plt.subplot(1, 2, 2)
        plt.plot(kills, label='Kills', color='limegreen', alpha=0.4)
        
        if len(kills) >= 50:
            avg_kills = np.convolve(kills, np.ones(50)/50, mode='valid')
            plt.plot(range(49, len(kills)), avg_kills, color='darkgreen', linewidth=2, label='Tendencia')
            
        plt.xlabel("Episodio")
        plt.ylabel("Enemigos Eliminados")
        plt.title("Eficacia de Combate")
        plt.legend()
        plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

# Detectar variables automáticamente
rewards_data = reward_history if 'reward_history' in globals() else []
kills_data = kill_history if 'kill_history' in globals() else []

if len(rewards_data) > 0:
    plot_rewards(rewards_data, kills_data)
else:
    print("⚠ No hay datos para graficar aún.")

In [ ]:
# CELDA 10: EVALUACIÓN PARA AGENTE RECURRENTE (GRU)
import imageio
import matplotlib.pyplot as plt
from collections import deque
import torch.nn.functional as F

def evaluate_recurrent_agent(env, model, episodes=3, max_steps=2100, stochastic=True, gif_name="eval_gru.gif"):
    print(f"🧠 Evaluando Agente con MEMORIA (GRU)...")
    model.eval()
    
    total_rewards = []
    frames_for_gif = []
    
    for ep in range(episodes):
        frame = env.reset()
        state_stack = deque([frame] * 4, maxlen=4)
        
        # --- CLAVE GRU: INICIALIZAR MEMORIA ---
        # Shape: (Num_Layers, Batch_Size, Hidden_Size) -> (1, 1, 512)
        hidden = torch.zeros(1, 1, 512).to(device)
        
        done = False
        step = 0
        ep_reward = 0
        
        if ep == 0: frames_for_gif.append(frame)
        print(f"--- Episodio {ep+1} ---")

        while not done and step < max_steps:
            step += 1
            
            # Preprocesamiento
            state_np = np.array(state_stack)
            x = torch.tensor(state_np, dtype=torch.float32).unsqueeze(0).to(device)
            
            with torch.no_grad():
                # --- CLAVE GRU: PASAR Y RECIBIR MEMORIA ---
                # Ahora recibimos 3 valores: logits, value, y la NUEVA memoria
                logits, _, new_hidden = model(x, hidden)
                
                # Actualizamos la memoria para el siguiente paso
                hidden = new_hidden

                if stochastic:
                    dist = torch.distributions.Categorical(logits=logits)
                    action = dist.sample().item()
                else:
                    action = torch.argmax(logits, dim=-1).item()

            # Environment Step
            next_frame, reward, done = env.step(action)
            state_stack.append(next_frame)
            ep_reward += reward
            
            if ep == 0: frames_for_gif.append(next_frame)

        total_rewards.append(ep_reward)
        print(f"   Terminado con Reward: {ep_reward:.2f}")

    # GIF
    if len(frames_for_gif) > 0:
        print(f"💾 Guardando GIF memoria: {gif_name}")
        rgb_frames = [np.stack([f]*3, axis=-1) for f in frames_for_gif]
        imageio.mimsave(gif_name, rgb_frames, fps=25)

    return total_rewards

# Ejecución
if 'agent' in globals():
    evaluate_recurrent_agent(env, agent, episodes=1, stochastic=True)
else:
    print("⚠ Define 'agent' primero.")

In [ ]:
# CELDA 11: random_agent + plot (sin cambios mayores)
def random_agent(env, num_eval=100):
    rewards = []
    for _ in range(num_eval):
        frame = env.reset()
        done = False
        ep_r = 0
        while not done:
            act = np.random.choice(5)
            _, r, done = env.step(act)
            ep_r += r
        rewards.append(ep_r)
    return rewards

def make_random_gif(env, gif_name="random_agent.gif"):
    frame = env.reset()
    frames = []
    done = False
    while not done:
        act = np.random.choice(5)
        next_frame, _, done = env.step(act)
        frames.append(next_frame)
    rgb_frames = [np.stack([f]*3, -1) for f in frames]
    imageio.mimsave(gif_name, rgb_frames, fps=20)
    print(f"GIF random: {gif_name}")

random_rewards = random_agent(env)
make_random_gif(env)

min_len = min(100, len(rewards))
plt.figure(figsize=(12,6))
plt.plot(rewards[-min_len:], label="PPO")
plt.plot(random_rewards[-min_len:], label="Random")
plt.title("PPO vs Random (últ 100 eps)")
plt.legend()
plt.grid(True)
plt.show()

Image(filename="random_agent.gif")